In [96]:
from lxml import html
import re
import requests

In [93]:
page = requests.get("https://wibu47.com/xem-phim/hellsing-ultimate/7")
page = html.fromstring(page.text)

In [103]:
def extract_mirrors(tree):
    # Get all <script> elements
    scripts = tree.xpath('//script')
    
    # Regex to match JS variables assigned non-empty URLs
    pattern = r'var\s+(\w+)\s*=\s*"((?:https?://)[^"]+)"'
    
    result = {}
    for script in scripts:
        script_text = script.text or ""  # handle empty scripts
        for var, url in re.findall(pattern, script_text):
            result[var] = url
    return result
extract_mirrors(page)

{'HYDRAX': 'https://playhydrax.com/?v=suY5x1ytrF',
 'VPRO': 'https://embed1.streamc.xyz/embed.php?hash=8bf5d7c0125eb75c785875ec0bd13153',
 'AHS': 'https://hls1.streamc.xyz/8bf5d7c0125eb75c785875ec0bd13153/hls.m3u8'}

In [108]:
def get_page_link(page:int):
    return f"https://wibu47.com/the-loai/anime?page={page}"
link = get_page_link(1)
page = requests.get(link).text
page = html.fromstring(page)

In [118]:
base_url = "https://wibu47.com"
def parse_anime_article(article):

    data = {}

    # link
    link = article.xpath('.//a/@href')
    data['link'] =base_url + link[0] if link else None

    # thumbnail image src
    img = article.xpath('.//img[contains(@class,"wp-post-image")]/@src')
    data['thumbnail'] = img[0] if img else None

    # episode
    episode = article.xpath('.//span[@class="mli-eps"]/text()')
    data['episode'] = episode[0].strip() if episode else None

    # rating
    rating = article.xpath('.//div[contains(@class,"anime-avg-user-rating")]/text()')
    data['rating'] = rating[0].strip() if rating else None

    # title
    title = article.xpath('.//h2[@class="Title"]/text()')
    data['title'] = title[0].strip() if title else None

    # views
    views = article.xpath('.//span[@class="Year"]/text()')
    data['views'] = views[0].replace('Lượt xem:','').strip() if views else None

    # quality, vote, time, date
    info = article.xpath('.//p[@class="Info"]')
    if info:
        info = info[0]
        q = info.xpath('.//span[@class="Qlty"]/text()')
        v = info.xpath('.//span[contains(@class,"Vote")]/text()')
        t = info.xpath('.//span[contains(@class,"Time")]/text()')
        d = info.xpath('.//span[contains(@class,"Date")]/text()')
        data['quality'] = q[0].strip() if q else None
        data['vote'] = v[0].strip() if v else None
        data['time'] = t[0].strip() if t else None
        data['date'] = d[0].strip() if d else None
    else:
        data['quality'] = data['vote'] = data['time'] = data['date'] = None

    # description
    desc = article.xpath('.//div[@class="Description"]/p[1]/text()')
    data['description'] = desc[0].strip() if desc else None

    # genre
    genre = article.xpath('.//p[contains(@class,"Genre")]/span/text()')
    data['genre'] = genre[0].replace('Thể loại:','').strip() if genre else None

    return data

In [120]:
def get_animes_in_page(page):
    return page.find_class("TPost C post-943 post type-post status-publish format-standard has-post-thumbnail hentry")

In [119]:
movie_list = page.find_class("TPost C post-943 post type-post status-publish format-standard has-post-thumbnail hentry")
movie = movie_list[0]
parse_anime_article(movie)

{'link': 'https://wibu47.com/phim/eiyuu-ou-bu-o-kiwameru-tame-tensei-su-soshite-sekai-saikyou-no-minarai-kishi/',
 'thumbnail': 'https://wibu47.com/images/3694.jpg',
 'episode': 'TẬP',
 'rating': '8.6',
 'title': 'Eiyuu-ou, bu o Kiwameru tame tensei su - Soshite, Sekai Saikyou no Minarai Kishi ♀~',
 'views': '33,408',
 'quality': 'HD',
 'vote': '9.3',
 'time': '06/12',
 'date': '2023',
 'description': 'Từ trên giường bệnh, Vua anh hùng Inglis đã nhìn xuống đế chế mà anh đã xây dựng bằng bàn tay hùng mạnh của mình mà đã cống hiến cả cuộc đời mình cho nghệ thuật quản lý nhà nước và hạnh phúc của thần dân. Ước nguyện duy nhất của anh ấy là được sống lại, cho chính anh ấy lần này bằng cách trao đổi linh hồn của mình vào tương lai. Nữ thần bảo trợ của anh, Alistia, nghe thấy lời cầu xin của anh và mỉm cười với anh. Tại buổi lễ trưởng thành đầu tiên của Inglis vào năm 6 tuổi, cô ấy được cho là không đủ tư cách để bắt đầu phong tước hiệp sĩ của mình. Tuy nhiên, với một phụ nữ có tham vọng như

In [215]:
def get_entry_point(info_page):
    panel = info_page.find_class("InfoList")[0]
    first_link = panel.xpath('//li[@class="AAIco-adjust latest_eps"]/a[@href]/@href')
    return first_link
def get_chapter_links(watch_page):
    panel = watch_page.find_class("list-episode tab-pane ABList")[0]
    links = panel.xpath('.//li/a[@href]')
    return [(a.text.strip() if a.text else '',base_url+  a.get('href')) for a in links]


def get_watch_link(film_detail_url:str, chapter):
    return film_detail_url.replace("phim", "xem-phim") + str(chapter)


In [223]:
def get_valid_watch_link(links):
    for link in links:
        if link[-1] != '/':
            return link
    return None

links = ['/xem-phim/mou-ippon/', '/xem-phim/mou-ippon/13', '/xem-phim/mou-ippon/12']
get_valid_watch_link(links)

'/xem-phim/mou-ippon/13'

In [228]:
import json

MAX_PAGE = 193
SAVE_EVERY_PAGES = 1
exception = []
episode_get_exception = []
movies = []

for page_num in range(MAX_PAGE):
    print(f"[INFO] Processing page {page_num+1}/{MAX_PAGE}")
    
    page = get_page_link(page_num)
    page = requests.get(page).text
    page = html.fromstring(page)
    animes = get_animes_in_page(page)
    
    for idx, anime in enumerate(animes):
        print(f"  [INFO] Anime {idx+1}/{len(animes)} on page {page_num+1}")
        info = parse_anime_article(anime)
        link = info['link']
        try:
            anime_page = requests.get(link).text
            anime_page = html.fromstring(anime_page)
            entry_points = get_entry_point(anime_page)
            entry_point = get_valid_watch_link(entry_points)
            if entry_point is None:
                exception.append({"error": "no entry_point", "link" : link})
                continue

            entry_point = base_url + entry_point
            entry_point = requests.get(entry_point).text
            entry_point = html.fromstring(entry_point)
            chapter_links = get_chapter_links(entry_point)
            episodes = []

            for episode_num, watch_link in chapter_links:
                episode = {}
                try:
                    watch_page = html.fromstring(requests.get(watch_link).text)
                    mirrors = extract_mirrors(watch_page)
                    episode["ep"] = episode_num
                    episode["mirrors"] = mirrors
                    episodes.append(episode)
                except Exception as e:
                    print(f"    [ERROR] Episode {episode_num} failed: {e}")
                    episode_get_exception.append({"error": str(e), "anime": link, "episode": episode_num})

            info['episodes'] = episodes
            movies.append(info)

            # Write progress after each anime
            with open("movies.json", "w", encoding="utf-8") as f:
                json.dump(movies, f, ensure_ascii=False, indent=2)

        except Exception as e:
            print(f"  [ERROR] Anime failed: {e}")
            exception.append({"error": str(e), "link": link})



[INFO] Processing page 1/193
  [INFO] Anime 1/20 on page 1
  [INFO] Anime 2/20 on page 1
  [INFO] Anime 3/20 on page 1
  [INFO] Anime 4/20 on page 1
  [INFO] Anime 5/20 on page 1
  [INFO] Anime 6/20 on page 1
  [INFO] Anime 7/20 on page 1
  [INFO] Anime 8/20 on page 1
  [INFO] Anime 9/20 on page 1
  [INFO] Anime 10/20 on page 1
  [INFO] Anime 11/20 on page 1
  [INFO] Anime 12/20 on page 1
  [INFO] Anime 13/20 on page 1
  [INFO] Anime 14/20 on page 1
  [INFO] Anime 15/20 on page 1
  [INFO] Anime 16/20 on page 1


KeyboardInterrupt: 

In [226]:
movies

[{'link': 'https://wibu47.com/phim/eiyuu-ou-bu-o-kiwameru-tame-tensei-su-soshite-sekai-saikyou-no-minarai-kishi/',
  'thumbnail': 'https://wibu47.com/images/3694.jpg',
  'episode': 'TẬP',
  'rating': '8.6',
  'title': 'Eiyuu-ou, bu o Kiwameru tame tensei su - Soshite, Sekai Saikyou no Minarai Kishi ♀~',
  'views': '33,408',
  'quality': 'HD',
  'vote': '9.3',
  'time': '06/12',
  'date': '2023',
  'description': 'Từ trên giường bệnh, Vua anh hùng Inglis đã nhìn xuống đế chế mà anh đã xây dựng bằng bàn tay hùng mạnh của mình mà đã cống hiến cả cuộc đời mình cho nghệ thuật quản lý nhà nước và hạnh phúc của thần dân. Ước nguyện duy nhất của anh ấy là được sống lại, cho chính anh ấy lần này bằng cách trao đổi linh hồn của mình vào tương lai. Nữ thần bảo trợ của anh, Alistia, nghe thấy lời cầu xin của anh và mỉm cười với anh. Tại buổi lễ trưởng thành đầu tiên của Inglis vào năm 6 tuổi, cô ấy được cho là không đủ tư cách để bắt đầu phong tước hiệp sĩ của mình. Tuy nhiên, với một phụ nữ có th